In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        self.fc1 = nn.Linear(16 *6 *6 , 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # ?
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)
        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [6]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [7]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0231,  0.0994, -0.0685, -0.0653,  0.1117,  0.0597, -0.0970,  0.0928,
          0.1246,  0.0191]], grad_fn=<AddmmBackward>)


In [8]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [17]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
loss

tensor(0.5232, grad_fn=<MseLossBackward>)

In [31]:
loss.backward()

In [32]:
net.conv1.weight.grad, net.conv1.bias.grad

(tensor([[[[ 0.0151, -0.0071,  0.0036],
           [ 0.0027,  0.0147, -0.0010],
           [-0.0046,  0.0021,  0.0061]]],
 
 
         [[[ 0.0072,  0.0004,  0.0141],
           [-0.0017, -0.0041, -0.0039],
           [-0.0027, -0.0125,  0.0018]]],
 
 
         [[[ 0.0129, -0.0094,  0.0143],
           [ 0.0059,  0.0130,  0.0036],
           [-0.0067,  0.0184, -0.0137]]],
 
 
         [[[-0.0033,  0.0069,  0.0061],
           [-0.0101, -0.0019, -0.0029],
           [ 0.0067,  0.0105,  0.0005]]],
 
 
         [[[ 0.0045, -0.0091, -0.0108],
           [ 0.0079, -0.0001, -0.0037],
           [ 0.0025,  0.0135, -0.0005]]],
 
 
         [[[ 0.0044,  0.0101, -0.0005],
           [-0.0095, -0.0006,  0.0001],
           [-0.0086,  0.0092, -0.0072]]]]),
 tensor([-0.0119,  0.0013,  0.0031, -0.0002,  0.0055, -0.0046]))

In [39]:
help([*net.parameters()][0].data.sub_)

Help on built-in function sub_:

sub_(...) method of torch.Tensor instance
    sub_(other, *, alpha=1) -> Tensor
    
    In-place version of :meth:`~Tensor.sub`



In [40]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [41]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()